In [1]:
from confluent_kafka import Producer, KafkaException
import mmh3

def banana_partitioner(key, partitions):
    if key is None or not isinstance(key, str):
        raise KafkaException("We expect all messages to have customer name as key")

    num_partitions = len(partitions)
    if key == "Banana":
        return partitions[-1]  # Banana will always go to the last partition

    # Other records will get hashed to the rest of the partitions
    return partitions[abs(mmh3.hash(key)) % (num_partitions - 1)]

# Example usage
conf = {
    'bootstrap.servers': '10.222.68.223:9092'
}

producer = Producer(conf)

In [2]:
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

# Produce messages with custom partitioning logic
def produce_message(topic, key, value):
    partitions = producer.list_topics().topics[topic].partitions.keys()
    partition = banana_partitioner(key, list(partitions))
    producer.produce(topic, key=key, value=value, partition=partition, callback=delivery_report)

In [3]:
produce_message('my_topic', 'Banana', 'Banana message')

KeyError: 'my_topic'

In [4]:
produce_message('my_topic', 'Apple', 'Apple message')

In [5]:
producer.flush()


Message delivered to my_topic [2]
Message delivered to my_topic [0]


0